# Multilabel classification on PASCAL using python data-layers

## 1. Setting

In [1]:
yolo_weights_path = "yolo/models/yolo-voc_old_asada_30000.weights"
test_data_folder_path = "text-renderer/data/MIT_en_ch/en/en_300/"#yolo/"
lexicon_path = "text-renderer/list/street_en_single_list-28.txt"

num_class = 28
input_size = (416,416)
batch_size = 1

## 2. Import library 

In [2]:
import sys
import matplotlib.pyplot as plt
sys.path.append("yolo/python")
import y2c
import caffe
import numpy as np
import os

## 3. Load Lexicon and Create test file 

In [3]:
lexicon = np.loadtxt(lexicon_path,dtype='str')
lexicon = np.array([l.rstrip("\n") for l in lexicon])
print "Lexicon:", lexicon

test_file_length = 0

with open(test_data_folder_path+"test.txt","w")as f:
    for i,j,k in os.walk(test_data_folder_path):
        for kk in k:
            path = os.path.join(i,kk)
            if "jpg" not in path:
                continue
            f.write(path+","+str(np.where(lexicon == path.split("/")[-2])[0][0])+"\n")
            test_file_length += 1
            
y2c.create_testing_file(test_data_folder_path+"test.txt", lexicon_path, batch_size, input_size,num_class)

Lexicon: ['FRAZZOLI' 'TEDRAKE' 'ASADA' 'BREAZEAL' 'DUBOWSKY' 'HERR' 'HOBURG' 'HOGAN'
 'IAGNEMMA' 'KAELBLING' 'LOZANO' 'KIM' 'REIF' 'WALTZ' 'PERAIRE' 'CHANDRA'
 'KASAN' 'MICALI' 'CHEN' 'HOSOI' 'RUS' 'HOW' 'ROY' 'SHAH' 'WILLIAMS'
 'LEONARD' 'BROOKS' 'KARAMAN']
------------------------------------------
Prototxt Created! Path: yolo/y2c_output_28.prototxt
------------------------------------------
Solver Created! Path: yolo/solver.prototxt
------------------------------------------


## 4. Transform the Model 

In [ ]:
y2c.transform(yolo_weights_path,"yolo/y2c_template.prototxt","yolo/target.caffemodel")

YOLO total weights :  67199109
------------------------------------------
Layers:  ['conv1', 'bn1', 'scale_conv1', 'conv2', 'bn2', 'scale_conv2', 'conv3', 'bn3', 'scale_conv3', 'conv4', 'bn4', 'scale_conv4', 'conv5', 'bn5', 'scale_conv5', 'conv6', 'bn6', 'scale_conv6', 'conv7', 'bn7', 'scale_conv7', 'conv8', 'bn8', 'scale_conv8', 'conv9', 'bn9', 'scale_conv9', 'conv10', 'bn10', 'scale_conv10', 'conv11', 'bn11', 'scale_conv11', 'conv12', 'bn12', 'scale_conv12', 'conv13', 'bn13', 'scale_conv13', 'conv14', 'bn14', 'scale_conv14', 'conv15', 'bn15', 'scale_conv15', 'conv16', 'bn16', 'scale_conv16', 'conv17', 'bn17', 'scale_conv17', 'conv18', 'bn18', 'scale_conv18', 'conv19', 'bn19', 'scale_conv19', 'conv20', 'bn20', 'scale_conv20', 'conv21', 'bn21', 'scale_conv21', 'conv_reg']
------------------------------------------
Model Matching: Success!!


## 5. Check Accuracy 

In [ ]:
solver = caffe.SGDSolver('yolo/solver.prototxt')
solver.net.copy_from('yolo/target.caffemodel')
solver.test_nets[0].share_with(solver.net)
acc = y2c.check_accuracy(solver.test_nets[0], test_file_length,64, 28)
print "Accuracy:", acc